In [1]:
!pip install cohere

INFO: pip is looking at multiple versions of pydantic to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 2.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 1.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.1 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.4/434.4 kB 4.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 4.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.14.6
    Uninstalling pydantic_core-2.14.6:
      Successfully uninstalled pydant

In [3]:
import cohere

co = cohere.Client('DbUauyI3zFWlrXUhOrZpB2yFZszSUs8wlmbZqGfl')

In [103]:
def generate_flashcards(text):
    summary_generated = co.generate(
        model='command-xlarge-nightly',
        prompt=f"Summarize the following text:\n\n{text}",
        max_tokens=150,
        num_generations=1,
        temperature=0.5
    )
    summary = summary_generated.generations[0].text.strip()
    questions_response = co.generate(
        model='command-xlarge-nightly',
        prompt=f"Create a list of questions from the following summary:\n\n{summary}",
        max_tokens=100,
        num_generations=5, 
        temperature=0.7
    )
    questions = [gen.text.strip() for gen in questions_response.generations]
    reranked_questions = co.rerank(
        query=f"Select the most relevant questions from the following list:",
        documents=questions, 
        top_n=3,  
        return_documents=True  
    )
    # for result in reranked_questions.results:
    #     print(f"Document: {result.document.text}")
    #     print(f"Relevance Score: {result.relevance_score}\n")
    best_result = max(reranked_questions.results, key=lambda x: x.relevance_score)
    text_with_questions = best_result.document.text
    selected_questions = [line.strip() for line in text_with_questions.splitlines() if line.startswith('-')]
    # for question in questions:
    #     print(question)
    flashcards = []
    for question in selected_questions:
        answer_response = co.generate(
            model='command-xlarge-nightly',
            prompt=f"Provide a concise answer to the following question based on the text: {text}\n\nQuestion: {question}",
            max_tokens=100,
            num_generations=1,  
            temperature=0.7
        )
        answer = answer_response.generations[0].text.strip()
        flashcards.append({
            "question": question,
            "answer": answer
        })

    return flashcards
    

In [105]:
text = 'Battle of Waterloo, (June 18, 1815), Napoleon’s final defeat, ending 23 years of recurrent warfare between France and the other powers of Europe. It was fought during the Hundred Days of Napoleon’s restoration, 3 miles (5 km) south of Waterloo village (which is 9 miles [14.5 km] south of Brussels), between Napoleon’s 72,000 troops and the combined forces of the duke of Wellington’s allied army of 68,000 (with British, Dutch, Belgian, and German units) and about 45,000 Prussians, the main force of Gebhard Leberecht von Blücher’s command.'
sum = generate_flashcards(text)
sum

[{'question': '- When and where did the Battle of Waterloo take place?',
  'answer': 'The Battle of Waterloo took place on June 18, 1815, about 3 miles south of Waterloo village, which is 9 miles south of Brussels.'},
 {'question': '- Who were the key military leaders involved in the Battle of Waterloo?',
  'answer': 'The key military leaders involved in the Battle of Waterloo were Napoleon, the duke of Wellington, and Gebhard Leberecht von Blücher.'},
 {'question': '- How many troops did Napoleon command at Waterloo?',
  'answer': 'Napoleon commanded 72,000 troops at Waterloo.'},
 {'question': '- Which nations were part of the coalition army led by the Duke of Wellington?',
  'answer': 'The coalition army led by the Duke of Wellington included British, Dutch, Belgian, and German units.'},
 {'question': '- What was the approximate size of the Prussian force at Waterloo?',
  'answer': 'The Prussian force at Waterloo was approximately 45,000 troops.'},
 {'question': "- What was the outco